#### CSC 180 Intelligent Systems 

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento




## Lab 3: Evaluating Neural Networks 


### Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Training with a Test Set with Early Stopping

**Overfitting** occurs when a neural network is trained to the point that it begins to memorize rather than generalize.  




### Split data into training and test using train_test_split

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# min delta is minimum improvement
# patience = 2 means wait for two epochs before stopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

# call back function triggers the early stopping function
model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)  


2024-09-16 08:38:08.838660: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/1000
4/4 - 1s - loss: 1.8933 - val_loss: 1.8676 - 1s/epoch - 311ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.8019 - val_loss: 1.7828 - 45ms/epoch - 11ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.7241 - val_loss: 1.6980 - 54ms/epoch - 13ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.6421 - val_loss: 1.6166 - 50ms/epoch - 13ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.5708 - val_loss: 1.5450 - 49ms/epoch - 12ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.5056 - val_loss: 1.4840 - 46ms/epoch - 12ms/step
Epoch 7/1000
4/4 - 0s - loss: 1.4384 - val_loss: 1.4162 - 49ms/epoch - 12ms/step
Epoch 8/1000
4/4 - 0s - loss: 1.3621 - val_loss: 1.3385 - 50ms/epoch - 12ms/step
Epoch 9/1000
4/4 - 0s - loss: 1.2951 - val_loss: 1.2732 - 49ms/epoch - 12ms/step
Epoch 10/1000
4/4 - 0s - loss: 1.2306 - val_loss: 1.2234 - 52ms/epoch - 13ms/step
Epoch 11/1000
4/4 - 0s - loss: 1.1905 - val_loss: 1.1899 - 50ms/epoch - 12ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.1706 - val_loss: 1.1675 - 46ms/epoch - 12ms/step
Epoch 13/1000
4/4 - 0s - l

Now that the neural network is trained, we can make predictions about the test set.  The following code predicts the type of iris for test set and displays the first five irises. 

In [5]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

2/2 [==============================] - 0s 4ms/step
[[5.32476744e-03 7.90230095e-01 2.04445094e-01]
 [9.85199153e-01 1.48007935e-02 1.26473410e-07]
 [4.71233164e-07 2.23572608e-02 9.77642298e-01]
 [3.59373982e-03 7.01048434e-01 2.95357883e-01]
 [7.70809036e-03 8.91285956e-01 1.01005897e-01]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set. 

## Saving Best Weights

It would be good idea to keep track of the most optimal weights during the entire training operation.  


An additional monitor, ModelCheckpoint,  is used and saves a copy of the neural network to **best_weights.hdf5** each time the validation score of the neural network improves.  

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [6]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

# saves the best model based on teh test error
checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

# saves the best model
model.load_weights('dnn/best_weights.hdf5') # load weights from best model

Epoch 1/1000
4/4 - 1s - loss: 3.6503 - val_loss: 3.6092 - 1s/epoch - 336ms/step
Epoch 2/1000
4/4 - 0s - loss: 3.4426 - val_loss: 3.4094 - 107ms/epoch - 27ms/step
Epoch 3/1000
4/4 - 0s - loss: 3.2561 - val_loss: 3.2167 - 111ms/epoch - 28ms/step
Epoch 4/1000
4/4 - 0s - loss: 3.0651 - val_loss: 3.0351 - 101ms/epoch - 25ms/step
Epoch 5/1000
4/4 - 0s - loss: 2.8834 - val_loss: 2.8623 - 77ms/epoch - 19ms/step
Epoch 6/1000
4/4 - 0s - loss: 2.7175 - val_loss: 2.6975 - 80ms/epoch - 20ms/step
Epoch 7/1000
4/4 - 0s - loss: 2.5576 - val_loss: 2.5424 - 91ms/epoch - 23ms/step
Epoch 8/1000
4/4 - 0s - loss: 2.4153 - val_loss: 2.3954 - 95ms/epoch - 24ms/step
Epoch 9/1000
4/4 - 0s - loss: 2.2762 - val_loss: 2.2580 - 86ms/epoch - 21ms/step
Epoch 10/1000
4/4 - 0s - loss: 2.1422 - val_loss: 2.1314 - 84ms/epoch - 21ms/step
Epoch 11/1000
4/4 - 0s - loss: 2.0196 - val_loss: 2.0132 - 91ms/epoch - 23ms/step
Epoch 12/1000
4/4 - 0s - loss: 1.9044 - val_loss: 1.9046 - 84ms/epoch - 21ms/step
Epoch 13/1000
4/4 - 0s 

### Potential Keras Issue on Small Networks Regarding Saving Optimal Weights

You might occasionally see this error:

```
OSError: Unable to create file (Unable to open file: name = 'dnn/best_weights.hdf5', errno = 22, error message = 'invalid argument', flags = 13, o_flags = 302)
```

Usually you can just run rerun the code and it goes away.  This is an unfortnuate result of saving a file each time the validation score improves (as described in the previous section).  If the errors improve two rapidly, you might try to save the file twice and get an error from these two saves overlapping.  For larger neural networks this will not be a problem because each training step will take longer, allowing for plenty of time for the previous save to complete.   

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [3]:
pred = model.predict(x_test)
pred

2/2 [==============================] - 0s 3ms/step


array([[0.30014408, 0.38217512, 0.31768072],
       [0.32052344, 0.28488693, 0.3945897 ],
       [0.2568317 , 0.52407473, 0.21909353],
       [0.3062607 , 0.34118366, 0.3525556 ],
       [0.28253448, 0.4571012 , 0.26036423],
       [0.31960025, 0.31521288, 0.36518687],
       [0.31127772, 0.32599732, 0.362725  ],
       [0.29980686, 0.35752395, 0.3426692 ],
       [0.27432838, 0.4926758 , 0.23299578],
       [0.30251512, 0.38282746, 0.31465742],
       [0.30588606, 0.30895847, 0.3851554 ],
       [0.32002023, 0.33005613, 0.3499237 ],
       [0.3192931 , 0.32507432, 0.3556326 ],
       [0.3203616 , 0.3226277 , 0.35701066],
       [0.3211702 , 0.22765332, 0.4511765 ],
       [0.30852264, 0.29936513, 0.39211217],
       [0.3033038 , 0.31543717, 0.38125905],
       [0.29999328, 0.3992403 , 0.30076647],
       [0.30802932, 0.33318576, 0.35878485],
       [0.30151188, 0.34233832, 0.3561498 ],
       [0.3232513 , 0.2770768 , 0.39967185],
       [0.3073789 , 0.31094012, 0.381681  ],
       [0.

In [5]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [6]:
y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 0.9736842105263158


In [7]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 0.9757085020242916


In [8]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 0.9736842105263158


In [21]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 1.0


In [9]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.91      0.95        11
           2       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


Calculating log loss

In [10]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions
[[4.62278072e-03 8.71348143e-01 1.24029100e-01]
 [9.98032272e-01 1.96734793e-03 3.99851984e-07]
 [2.72625257e-05 1.96773782e-02 9.80295360e-01]
 [5.12181176e-03 8.19644332e-01 1.75233886e-01]
 [3.26915109e-03 8.95215034e-01 1.01515815e-01]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.09630766139659834


## Evaluating Regression Models

Regression results are evaluated differently than classification.  Consider the following code. 

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']

# SPECIFY that 1 is the axis
df.drop('name',axis=1,inplace=True)

missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron 


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 2s - loss: 378535.2500 - val_loss: 615.1457 - 2s/epoch - 153ms/step
Epoch 2/1000
10/10 - 0s - loss: 37723.9883 - val_loss: 63643.7539 - 64ms/epoch - 6ms/step
Epoch 3/1000
10/10 - 0s - loss: 28512.2012 - val_loss: 114.4837 - 73ms/epoch - 7ms/step
Epoch 4/1000
10/10 - 0s - loss: 5421.6133 - val_loss: 8275.8623 - 71ms/epoch - 7ms/step
Epoch 5/1000
10/10 - 0s - loss: 3509.1045 - val_loss: 200.3689 - 68ms/epoch - 7ms/step
Epoch 6/1000
10/10 - 0s - loss: 1110.8137 - val_loss: 991.4215 - 68ms/epoch - 7ms/step
Epoch 7/1000
10/10 - 0s - loss: 404.7865 - val_loss: 289.1327 - 68ms/epoch - 7ms/step
Epoch 8/1000
10/10 - 0s - loss: 365.9644 - val_loss: 149.6090 - 59ms/epoch - 6ms/step
Epoch 8: early stopping


### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [12]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

Final score (MSE): 18.79427146911621


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [13]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 4.335236072540283


# Performance Improvement by Normalizing Features and Tuning Hyperparameters

There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [14]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',1,inplace=True)


# normalize all input features
if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 0s - loss: 605.9440 - val_loss: 556.0004
Epoch 2/1000
10/10 - 0s - loss: 566.8333 - val_loss: 507.1982
Epoch 3/1000
10/10 - 0s - loss: 505.1319 - val_loss: 429.8139
Epoch 4/1000
10/10 - 0s - loss: 409.0138 - val_loss: 316.9863
Epoch 5/1000
10/10 - 0s - loss: 273.1548 - val_loss: 177.2618
Epoch 6/1000
10/10 - 0s - loss: 125.7235 - val_loss: 61.3011
Epoch 7/1000
10/10 - 0s - loss: 38.0301 - val_loss: 39.0215
Epoch 8/1000
10/10 - 0s - loss: 36.1851 - val_loss: 35.2568
Epoch 9/1000
10/10 - 0s - loss: 26.0564 - val_loss: 23.3026
Epoch 10/1000
10/10 - 0s - loss: 21.4498 - val_loss: 19.8781
Epoch 11/1000
10/10 - 0s - loss: 18.4882 - val_loss: 16.6677
Epoch 12/1000
10/10 - 0s - loss: 16.2612 - val_loss: 14.6550
Epoch 13/1000
10/10 - 0s - loss: 14.9690 - val_loss: 12.8048
Epoch 14/1000
10/10 - 0s - loss: 13.7016 - val_loss: 11.0336
Epoch 15/1000
10/10 - 0s - loss: 12.8530 - val_loss: 9.9106
Epoch 16/1000
10/10 - 0s - loss: 12.1488 - val_loss: 8.9551
Epoch 17/1000
10/10 - 0s

In [15]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 2.166762590408325


In [16]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

,ground_truth,predicted
0,33.0,32.868351
1,28.0,30.512417
2,19.0,20.677746
3,13.0,15.107977
4,14.0,13.358086
...,...,...
75,19.9,21.390100
76,17.5,17.585392
77,28.0,30.204233
78,29.0,29.141594
